In [0]:
# %% [code]
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# %% [code]
import numpy as np
import pandas as pd
import os
import json
import re
from pandas import ExcelWriter
from pandas import ExcelFile

# %% [code]
custom_license_dir = '/kaggle/input/CORD-19-research-challenge/custom_license/custom_license/'

noncomm_use_subset_dir = '/kaggle/input/CORD-19-research-challenge/noncomm_use_subset/noncomm_use_subset/'

biorxiv_medrxiv_dir = '/kaggle/input/CORD-19-research-challenge/biorxiv_medrxiv/biorxiv_medrxiv/'

comm_use_subset = '/kaggle/input/CORD-19-research-challenge/comm_use_subset/comm_use_subset/'

# %% [code]
#List and print all files names in a directory

filenames_custom_license_dir = os.listdir(custom_license_dir)
# print("custom_license_dir")
# print("Number of articles retrieved from noncomm_use_subset:", len(filenames_custom_license_dir))
# print(filenames_custom_license_dir)


filenames_noncomm_use_subset_dir = os.listdir(noncomm_use_subset_dir)
# print("noncomm_use_subset_dir")
# print("Number of articles retrieved from noncomm_use_subset:", len(filenames_noncomm_use_subset_dir))
# print(filenames_noncomm_use_subset_dir)


filenames_biorxiv_medrxiv_dir = os.listdir(biorxiv_medrxiv_dir)
# print("biorxiv_medrxiv_dir")
# print("Number of articles retrieved from biorxiv_medrxiv_dir:", len(filenames_biorxiv_medrxiv_dir))
# print(filenames_biorxiv_medrxiv_dir)


filenames_comm_use_subset_dir = os.listdir(comm_use_subset)
# print("comm_use_subset")
# print("Number of articles retrieved from comm_use_subset:", len(filenames_comm_use_subset_dir))
# print(filenames_comm_use_subset_dir)

# %% [code]
#Load all files

all_files_custom_license_dir = []

for filename in filenames_custom_license_dir:
    filename = custom_license_dir + filename
    file = json.load(open(filename, 'rb'))
    all_files_custom_license_dir.append(file)


all_files_noncomm_use_subset_dir = []

for filename in filenames_noncomm_use_subset_dir:
    filename = noncomm_use_subset_dir + filename
    file = json.load(open(filename, 'rb'))
    all_files_noncomm_use_subset_dir.append(file)

    
all_files_biorxiv_medrxiv_dir = []

for filename in filenames_biorxiv_medrxiv_dir:
    filename = biorxiv_medrxiv_dir + filename
    file = json.load(open(filename, 'rb'))
    all_files_biorxiv_medrxiv_dir.append(file)

    
all_files_comm_use_subset_dir = []

for filename in filenames_comm_use_subset_dir:
    filename = comm_use_subset + filename
    file = json.load(open(filename, 'rb'))
    all_files_comm_use_subset_dir.append(file)

# %% [code]
#Print number of files loaded from each directory

print(len(all_files_custom_license_dir))

print(len(all_files_noncomm_use_subset_dir))

print(len(all_files_biorxiv_medrxiv_dir))

print(len(all_files_comm_use_subset_dir))

# %% [code]
# Combine in single dataframe

all_files = [all_files_custom_license_dir, all_files_noncomm_use_subset_dir, all_files_biorxiv_medrxiv_dir, all_files_comm_use_subset_dir]
print(len(all_files))

# %% [code]
file = all_files[0][0]
print("Dictionary keys:", file.keys())

# %% [code]
# Read section data file
section_data = pd.read_excel('/kaggle/input/covidsectionheads/covid_section_heads.xlsx', sheet_name='Sheet1')
section_data.columns = ['section','frequency','category']
section_data.head(2)

# %% [code]
# List section categories of interest
introduction_categories = ["Introduction"]
discussion_categories = ["Discussion", "Results and Discussion"]
conclusion_categories = ["Results", "Results and Discussion"]


# Create sub-lists
introduction_list_temp = section_data[section_data['category'].isin(introduction_categories)]
introduction_list = list(introduction_list_temp['section'])
#print(introduction_list)


discussion_list_temp = section_data[section_data['category'].isin(discussion_categories)]
discussion_list = list(discussion_list_temp['section'])
#print(discussion_list)


conclusion_list_temp = section_data[section_data['category'].isin(conclusion_categories)]
conclusion_list = list(conclusion_list_temp['section'])
#print(conclusion_list)

# %% [code]
#Refine sub-lists (remove non-alphanumeric characters)

introduction_clean_list = [None]*len(introduction_list)
for value in range(len(introduction_list)):
    value_tf = str(introduction_list[value]).replace(" ","_")
    value_tf = re.sub(r'\W+','', value_tf)
    value_tf = value_tf.replace("_", " ")
    value_tf = value_tf.strip()
    introduction_clean_list[value] = value_tf


discussion_clean_list = [None]*len(discussion_list)
for value in range(len(discussion_list)):
    value_tf = str(discussion_list[value]).replace(" ","_")
    value_tf = re.sub(r'\W+','', value_tf)
    value_tf = value_tf.replace("_", " ")
    value_tf = value_tf.strip()
    discussion_clean_list[value] = value_tf


conclusion_clean_list = [None]*len(conclusion_list)
for value in range(len(conclusion_list)):
    value_tf = str(conclusion_list[value]).replace(" ","_")
    value_tf = re.sub(r'\W+','', value_tf)
    value_tf = value_tf.replace("_", " ")
    value_tf = value_tf.strip()
    conclusion_clean_list[value] = value_tf

# %% [code]
#Extract title

def findtitle_file(file):
    metadatalength = len(file['metadata'])
    if metadatalength > 0:
        if "title" in file['metadata'].keys():
            titlelength = len(file['metadata']['title'])
            cleaneduptext = ""
            cleaneduptext += file['metadata']['title']
            return cleaneduptext

findtitle_file(all_files[0][30])

# %% [code]
#Extract abstract

def findabstract_file(file):
    cleaneduptext  = ""
    
    abstractlength = len(file['abstract']) 
    for paragraph in range(abstractlength):
        cleaneduptext += file['abstract'][paragraph]['text']      
    return cleaneduptext

findabstract_file(all_files[1][0])

# %% [code]
#Extract introduction

def findintroduction_file(file, introduction_clean_list):
    bodytextlength = len(file['body_text'])
    cleaneduptext  = ""
    
#     print(cleaneduptext)
    for paragraph in range(bodytextlength):
#         print("Entered outer loop")
        searchspace = file['body_text'][paragraph]['section']
#         print(searchspace)
        for inputstring in introduction_clean_list:
#             print("Entered inner loop")
#             print(inputstring)
            if inputstring.upper() == searchspace.upper(): 
                cleaneduptext += file['body_text'][paragraph]['text']
    
    cleaneduptext = re.sub(r'\([^)]*\)', '', cleaneduptext)
    cleaneduptext = re.sub(r'\[[^)]*\]', '', cleaneduptext)    
    return cleaneduptext

findintroduction_file(all_files[1][2], introduction_clean_list)

# %% [code]
#Extract discussion

def finddiscussion_file(file, discussion_clean_list):
    bodytextlength = len(file['body_text'])
    cleaneduptext  = ""
    
    for paragraph in range(bodytextlength):
        searchspace = file['body_text'][paragraph]['section']
        for inputstring in discussion_clean_list:
            if inputstring.upper() == searchspace.upper(): 
                cleaneduptext += file['body_text'][paragraph]['text']

    cleaneduptext = re.sub(r'\([^)]*\)', '', cleaneduptext)
    cleaneduptext = re.sub(r'\[[^)]*\]', '', cleaneduptext)
    return cleaneduptext

finddiscussion_file(all_files[1][2], discussion_clean_list)

# %% [code]
#Extract conclusion

def findconclusion_file(file, conclusion_clean_list):
    bodytextlength = len(file['body_text'])

    cleaneduptext  = ""
    
    for paragraph in range(bodytextlength):
        searchspace = file['body_text'][paragraph]['section']
        for inputstring in conclusion_clean_list:
            if inputstring.upper() == searchspace.upper(): 
                cleaneduptext += file['body_text'][paragraph]['text']

    cleaneduptext = re.sub(r'\([^)]*\)', '', cleaneduptext)
    cleaneduptext = re.sub(r'\[[^)]*\]', '', cleaneduptext)      
    return cleaneduptext

findconclusion_file(all_files[1][2], conclusion_clean_list)

# %% [code]
cleaned_files = []

for directory in range(len(all_files)):
    for file in all_files[directory]:
        features = [
            file['paper_id'],
            findtitle_file(file),
            findabstract_file(file),
            findintroduction_file(file, introduction_clean_list),
            finddiscussion_file(file, discussion_clean_list),
            findconclusion_file(file, conclusion_clean_list),            
            (findintroduction_file(file, introduction_clean_list)+finddiscussion_file(file, discussion_clean_list)+findconclusion_file(file, conclusion_clean_list))
        ]
        cleaned_files.append(features)

# %% [code]
col_names = [
    'paper_id', 
    'title',
    'abstract',
    'introduction',
    'discussion',
    'conclusion',
    'body'
]

clean_df = []
clean_df = pd.DataFrame(cleaned_files, columns=col_names)
print(len(clean_df))
clean_df.head()

# %% [code]
# Identify if papers have sections
summary = []

count_no_introduction = 0
count_no_discussion = 0
count_no_conclusion = 0
count_no_body = 0

for row in range(len(clean_df)):
    if len(clean_df['introduction'][row])>0:
        has_introduction = 1
    else:
        has_introduction = 0
        count_no_introduction += 1
    if len(clean_df['discussion'][row])>0:
        has_discussion = 1
    else:
        has_discussion = 0
        count_no_discussion += 1
    if len(clean_df['conclusion'][row])>0:
        has_conclusion = 1
    else:
        has_conclusion = 0
        count_no_conclusion += 1
    if has_introduction == 0 and has_discussion == 0 and has_conclusion == 0:
        has_body = 0
        count_no_body += 1
    else:
        has_body = 1
    features = [
        clean_df['paper_id'][row],
        has_introduction,
        has_discussion,
        has_conclusion,
        has_body
    ]
    summary.append(features)

col_names = [
    'paper_id', 
    'has_introduction',
    'has_discussion',
    'has_conclusion',
    'has_body'
]    
   
clean_summary = []
clean_summary = pd.DataFrame(summary, columns=col_names)


print("No body in "+str(count_no_body))
print("No introduction in "+str(count_no_introduction))
print("No discussion in "+str(count_no_discussion))
print("No conclusion in "+str(count_no_conclusion))

# %% [code]
clean_summary[clean_summary['has_body']==0]['paper_id']

# %% [code]
list(clean_summary[clean_summary['has_body']==0]['paper_id'])

# %% [code]
# delete_indices = clean_df[clean_df['paper_id'].isin(list(clean_summary[clean_summary['has_body']==0]['paper_id']))].index

# clean_df.drop(delete_indices , inplace=True)
# len(clean_df)

# %% [code]
metadata = pd.read_csv('/kaggle/input/CORD-19-research-challenge/metadata.csv', usecols = ['sha', 'source_x'])

# %% [code]
metadata.head()

# %% [code]
clean_df = clean_df.merge(metadata, how = 'left', left_on = 'paper_id', right_on = 'sha')

# %% [code]
clean_df.head()

# %% [code]
clean_df.drop(columns=['sha'], inplace = True)

# %% [code]
clean_df.rename(columns={"source_x": "source"}, inplace = True)

clean_df.head()

# %% [code]
clean_df.to_csv('/kaggle/working/CORD-19.csv', index=False)

# %% [code]
!ls

# %% [code]
!java -version

# %% [code]
!wget -O solr-8.5.0.zip "https://archive.apache.org/dist/lucene/solr/8.5.0/solr-8.5.0.zip"

# %% [code]
!unzip solr-8.5.0.zip

# %% [code]
!ls

# %% [code]
!pip install pysolr
!pip install kazoo #Needed for python zookeeper

# %% [code]
!solr-8.5.0/bin/solr start -force

# %% [code]
!solr-8.5.0/bin/solr create -c covid19 -s 1 -rf 1 -force

# %% [code]
# Using _default configset with data driven schema functionality. NOT RECOMMENDED for production use.
!solr-8.5.0/bin/solr config -c covid19 -p 8983 -action set-user-property -property update.autoCreateFields -value false

# %% [code]
!curl -X POST -H 'Content-type:application/json' --data-binary '{"add-field-type" : {"name":"keywordText","class":"solr.TextField", "positionIncrementGap":"100", "indexAnalyzer" : {"tokenizer":{"class":"solr.KeywordTokenizerFactory" }, "filters":[{"class":"solr.TrimFilterFactory"},{"class":"solr.StopFilterFactory", "ignoreCase":true, "words":"lang/stopwords_en.txt"},{"class":"solr.ManagedSynonymGraphFilterFactory", "managed":"english" },{"class":"solr.RemoveDuplicatesTokenFilterFactory"},{"class":"solr.FlattenGraphFilterFactory"}]},"queryAnalyzer" : {"tokenizer":{"class":"solr.KeywordTokenizerFactory" },"filters":[{"class":"solr.TrimFilterFactory"},{"class":"solr.StopFilterFactory", "ignoreCase":true, "words":"lang/stopwords_en.txt"},{"class":"solr.ManagedSynonymGraphFilterFactory", "managed":"english" },{"class":"solr.RemoveDuplicatesTokenFilterFactory"}]}}}' http://localhost:8983/solr/covid19/schema

# %% [code]
!curl -X POST -H 'Content-type:application/json' --data-binary '{"add-field": {"name":"title", "type":"text_en_splitting", "multiValued":false, "stored":true, "indexed":true}}' http://localhost:8983/solr/covid19/schema;
!curl -X POST -H 'Content-type:application/json' --data-binary '{"add-field": {"name":"abstract", "type":"text_en_splitting", "multiValued":false, "stored":true, "indexed":true}}' http://localhost:8983/solr/covid19/schema;
!curl -X POST -H 'Content-type:application/json' --data-binary '{"add-field": {"name":"text_body", "type":"text_en_splitting", "multiValued":false, "stored":true, "indexed":true}}' http://localhost:8983/solr/covid19/schema;
!curl -X POST -H 'Content-type:application/json' --data-binary '{"add-field": {"name":"source", "type":"keywordText", "multiValued":false, "stored":true, "indexed":true}}' http://localhost:8983/solr/covid19/schema;

# %% [code]


# %% [code]
import pysolr
solr = pysolr.Solr('http://localhost:8983/solr/covid19/', timeout=10)

# %% [code]
solr_content = {}
solr_content['id'] = 'id-9876'
solr_content['title'] = 'COVID-19 epidemic'
solr_content['source'] = 'Source-2'
solr_content['abstract'] = 'COVID-19 Epidemic causing tensions'
solr_content['text_body'] = 'Acute lung injury is a marked effect of the COVID-19 patients.'
solr.add([solr_content])
solr.commit()

# %% [code]
# Search for data
results = solr.search("\"acute lung injury\"", **{
    'hl': 'true',
    'hl.fragsize': 100,
    'hl.bs.type': 'SENTENCE',
    'hl.method' : 'unified',
    'hl.snippets' : 5,
    'df':"text_body"
})

print("Saw {0} result(s).".format(len(results)))

for result in results:
    print('TITLE : ', result['title'])
    #display(HTML('<h1>' + result['title'][0]) + '</h1>')
    hl_snippets = results.highlighting[result['id']]['text_body']
    
    for index, snippet in enumerate(hl_snippets, start=1):
        print('\t\t', index, '. ', snippet, '\n')
    print('############################')